In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta,time
from streamlit_gsheets import GSheetsConnection
from auxiliar.auxiliar import *
from auxiliar.omie_aux import *

In [ ]:
list_os = pegar_todos_os()

In [ ]:
list_os

In [ ]:
len(list_os)

# Subir base

In [ ]:
clientes_omie = atualizar_base_clientes()

In [ ]:
subir_dados_mongodb("id_clientes",clientes_omie)

In [ ]:
deletar_todos_documentos("id_clientes")

# testar subir OS